# 2020-05-14 스터디 대체과제
<br>
세현이가 구해준 데이터를 RDD를 통해 처리해보기
#### Q. 현재 전시회 기준으로 레코드가 작성되어있는 csv 파일을 아티스트의 아이디 기준으로 전시회를 나열하고 싶다. 
#### 그리고 전시회마다 타입이 있는데 이 타입별로 정렬해보기

#####  ex )
<img src="./study_result_image.png" width=600>
<br>
##### data info
- id : 전시회 고유 아이디
- title : 전시회 이름(딱히 없어도 됨; 아이디로 나열 ㄱ)
- type : 전시회 타입 (S, G, F, B, nan)
- artitsts : 전시회에 참여한 작가들의 아이디와 이름이 딕셔너리 배열로 저장되어 있음
- city, country, institution, begin_date, end_date, curators 는 버립시다
<br><br>
- 굳이 Null 데이터는 없어도 되니 알아서들 잘 날려서 해봅시당

In [1]:
import pandas as pd

### 데이터 불러오기

In [2]:
alldata = pd.read_csv("./exhibition_f1to3.csv")
alldata.columns = ['id','type','title','city','country','institution','begin_date','end_date','artists','curators']

In [3]:
alldata.head(10)

,id,type,title,city,country,institution,begin_date,end_date,artists,curators
0,100005,S,Nadir Afonso,172.0,13.0,4293.0,2002-02-01,2002-04-01,"[{""id"": ""48940"", ""name"": ""Nadir Afonso""}]",NaN
1,100009,G,Au delà du Copan,63.0,15.0,8282.0,2005-11-03,2005-12-02,"[{""id"": ""26524"", ""name"": ""Fabiano Marques""}, {...",NaN
2,100014,S,Guy Limone,63.0,15.0,8282.0,2006-06-02,2006-07-14,"[{""id"": ""15717"", ""name"": ""Guy Limone""}]",NaN
3,100019,F,bâlelatina 2008,112.0,3.0,6463.0,2008-06-03,2008-06-08,NaN,NaN
4,100026,S,Donald Judd,2.0,14.0,10179.0,2003-02-25,2003-05-16,"[{""id"": ""2013"", ""name"": ""Donald Judd""}]",NaN
5,100031,G,insubstantial pageant faded,398.0,18.0,9915.0,2007-09-21,2007-12-21,"[{""id"": ""4704"", ""name"": ""Martin Creed""}, {""id""...",NaN
6,100036,S,Julie Mehretu: City Sitings,471.0,18.0,2449.0,2007-11-23,2008-03-30,"[{""id"": ""18271"", ""name"": ""Julie Mehretu""}]",NaN
7,100041,S,Chloe Piene,12.0,1.0,23127.0,2007-11-03,2007-12-22,"[{""id"": ""8902"", ""name"": ""Chloe Piene""}]",NaN
8,100047,S,Early Sculpture,2.0,14.0,10179.0,2005-06-22,2005-07-30,"[{""id"": ""5646"", ""name"": ""Robert Morris""}]",NaN
9,100051,S,Joseph Kosuth,2.0,14.0,10179.0,2006-02-08,2006-04-13,"[{""id"": ""556"", ""name"": ""Joseph Kosuth""}]",NaN


In [4]:
data = alldata[["id", "type", "artists"]]
data = data.dropna()

### sample
19만 레코드가 넘어가니 샘플로 먼저 해보고 전체데이터로 해봅시다

In [7]:
sample = data.head(30)

In [8]:
srdd = spark.createDataFrame(sample).rdd

### all data

In [5]:
srdd = spark.createDataFrame(data).rdd

### 사용자 정의 함수

In [10]:
def fun(row):
    li = []
    for r in row[1]:
        li.append((row[0], r["id"], r["name"]))
    return li

In [11]:
def sorttype(row):
#     sorted_dict = {'S':[], 'F':[], 'G':[], 'B':[], 'nan':[]}
    sorted_dict = dict()
    for r in row:
        sorted_dict[r[0]] = sorted_dict.get(r[0], []) + [r[1]]
#         sorted_dict[r[0]] = sorted_dict.get(r[0], "") + " " + str(r[1])
    return sorted_dict

### map,reduce

In [17]:
resultrdd = srdd.map(lambda row : ((row[1], row[0]), eval(row[2])))\
.flatMap(lambda row : fun(row))\
.map(lambda row : ((row[1], row[2]), row[0]))\
.groupByKey()\
.map(lambda row : (row[0][0], row[0][1], sorttype(list(row[1]))))\
.collect()

### dataframe 화

In [13]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

schema = StructType([StructField("artist_id", StringType(), True), StructField("artist_name", StringType(), True), StructField("titles", StringType(), True)])

df = sqlContext.createDataFrame(resultrdd, schema)

In [14]:
# toPandas() : spark 데이터프레임을 pandas 데이터프레임으로
pd_df = df.select("*").toPandas()

In [15]:
pd_df

,artist_id,artist_name,titles
0,48940,Nadir Afonso,{S=[100005]}
1,41476,Rubens Mano,{G=[100009]}
2,66156,Donatella Spaziani,{G=[100009]}
3,109784,Raquel Garbelotti,{G=[100009]}
4,163369,Emmanuelle Rapin,{G=[100009]}
5,163370,Nicolas Richard,{G=[100009]}
6,163372,Ana Teixeira,{G=[100009]}
7,2013,Donald Judd,"{S=[100026], G=[100040]}"
8,10831,Rachel Harrison,{G=[100031]}
9,11721,Trisha Donnelly,{G=[100031]}


In [291]:
pd_df.to_csv("./preprocessed_by_artist.csv")

<img src="./ㅜㅠ.png" width=200>